In [1]:
# NOTES
# RUN CELLS THROUGH TO THE CELL LABELED "DRIVER CODE" IN ORDER

In [2]:
import sklearn as sk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import scipy.io as sio
from mat4py import loadmat 
# from keras.utils import to_categorical
import time
import glob
# import tensorflow as tf
# from sklearn.model_selection import train_test_split
!ls

Channel-0.png                        Channel-6.png
Channel-1.png                        Channel-7.png
Channel-2.png                        P300 Analysis.ipynb
Channel-3.png                        P300-12-08-2021-ResponseGraphs.ipynb
Channel-4.png                        mbsi-p300-dataset
Channel-5.png                        p300 raw vs. butter filter.jpg


In [3]:
folder = './mbsi-p300-dataset/*.mat'
files = glob.glob(folder)
files.pop(3) #drop the data badly structured
print(files)
# test_file = files.pop(0) #last subject for testing the network

['./mbsi-p300-dataset/P300S02.mat', './mbsi-p300-dataset/P300S03.mat', './mbsi-p300-dataset/P300S01.mat', './mbsi-p300-dataset/P300S05.mat', './mbsi-p300-dataset/P300S07.mat', './mbsi-p300-dataset/P300S06.mat', './mbsi-p300-dataset/P300S08.mat']


In [4]:
from scipy.signal import butter, lfilter, freqz

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y


# Filter requirements.
order = 6
fs = 24.0       # sample rate, Hz
cutoff = 1.5  # desired cutoff frequency of the filter, Hz

# Get the filter coefficients so we can check its frequency response.
b, a = butter_lowpass(cutoff, fs, order)

print(a)

[ 1.         -4.48456301  8.52900508 -8.77910797  5.14764268 -1.62771478
  0.21658286]


In [5]:
def data_to_array(): 
    unfiltered_data = []
    filtered_data = []

    for file in files:
        raw_data = loadmat(file)
        useful_data = raw_data['data'].copy()
        print(file)
        X = np.array(useful_data['X'])
        X_filtered = butter_lowpass_filter(X, cutoff, fs, order)
        Y = useful_data['y']
        T = useful_data['trial']
        F = useful_data['flash']

        xlen = len(X)
        for idx in F:
            if idx[3] == 2:
                if not len(unfiltered_data):
                    for electrode in range(8):
                        unfiltered_data.append(np.array(X[idx[0]-50:idx[0]+250, electrode]))
                        filtered_data.append(np.array(X[idx[0]-50:idx[0]+250, electrode]))
                else:
                    for electrode in range(8):
                        if not (idx[0]-50 < 0 or idx[0]+250 > xlen):
                            unfiltered_data[electrode] = np.vstack((unfiltered_data[electrode], np.array(X[idx[0]-50:idx[0]+250, electrode])))
                            filtered_data[electrode] = np.vstack((filtered_data[electrode], np.array(X[idx[0]-50:idx[0]+250, electrode])))
    
    unfiltered_means = [np.mean(i, axis = 0) for i in unfiltered_data]
    filtered_means = [np.mean(i, axis = 0) for i in filtered_data]
    return unfiltered_means, filtered_means

In [6]:
def subplot_p300(arr1, arr2):
    graph_axis = [i/250 for i in range(-50, 250) ]
    rows = len(arr1)
    cols = 2    
    fig, axes = plt.subplots(rows, cols, figsize=(25,60))
    
    for i in range(rows):
        axes[i][0].plot(graph_axis, arr1[i])
        axes[i][0].set_title(f"Arr1 Channel #{i}")
        axes[i][1].plot(graph_axis, arr1[2])
        axes[i][1].set_title(f"Arr2 Channel #{i}") 

    
    plt.savefig('p300 raw vs. butter filter.jpg', bbox_inches='tight',dpi=150 )
    plt.show()    

In [ ]:
# driver code 
unfiltered, filtered =  data_to_array()

subplot_p300(unfiltered, filtered)



# X, Y, Trials, Flash = visualise_data()

./mbsi-p300-dataset/P300S02.mat
./mbsi-p300-dataset/P300S03.mat
./mbsi-p300-dataset/P300S01.mat
./mbsi-p300-dataset/P300S05.mat
./mbsi-p300-dataset/P300S07.mat
./mbsi-p300-dataset/P300S06.mat
./mbsi-p300-dataset/P300S08.mat


In [ ]:
# BELOW IS ARCHIVE

In [9]:
# def visualise_p300(arr):
    
# #     print(means_arr[0].shape)
#     graph_axis = [i/250 for i in range(-50, 250) ]        
    
#     for i in range(len(means_arr)):
#         plt.figure(facecolor=(1, 1, 1))
#         plt.plot(graph_axis,means_arr[i])
#         y_lab = f'Channel: {i}'
#         plt.title(f'Channel: {i}')
#         plt.ylabel(y_lab)
#         plt.xlabel(time)
#         plt.savefig(f'Channel-{i}.png')
#         plt.show()
# #     return X_filtered, Y, T, F

In [8]:
# def data_to_array(filter_flag): 
#     electrode_measurements = []
#     #     file = files[0];
#     for file in files:
#         raw_data = loadmat(file)
#         useful_data = raw_data['data'].copy()
#         print(file)
#         X = np.array(useful_data['X'])
#         X = butter_lowpass_filter(X, cutoff, fs, order) if filter_flag else X
#         Y = useful_data['y']
#         T = useful_data['trial']
#         F = useful_data['flash']

#     #         X_filtered = butter_lowpass_filter(X, cutoff, fs, order)
#     #         [250]

#         i=0
#         xlen = len(X)
#         for idx in F:
#             if idx[3] == 2:
#                 i += 1
#                 if not len(electrode_measurements):
#                     for electrode in range(8):
#                         electrode_measurements.append(np.array(X[idx[0]-50:idx[0]+250, electrode]))
#                 else:
#                     for electrode in range(8):
#                         if not (idx[0]-50 < 0 or idx[0]+250 > xlen):
#                             electrode_measurements[electrode] = np.vstack((electrode_measurements[electrode], np.array(X[idx[0]-50:idx[0]+250, electrode])))
    
#     means_arr = [np.mean(i, axis = 0) for i in electrode_measurements]
#     return means_arr

In [7]:
# def visualise_data():
#     electrode_measurements = []
#     #     file = files[0];
#     for file in files:
#         raw_data = loadmat(file)
#         useful_data = raw_data['data'].copy()
#         print(file)
#         X = np.array(useful_data['X'])
#         X_filtered = butter_lowpass_filter(X, cutoff, fs, order)
#         Y = useful_data['y']

#         T = useful_data['trial']
#         F = useful_data['flash']

#     #         X_filtered = butter_lowpass_filter(X, cutoff, fs, order)
#     #         [250]

#         i=0
#         xlen = len(X)
#         for idx in F:
#             if idx[3] == 2:
#                 i += 1
#                 if not len(electrode_measurements):
#                     for electrode in range(8):
#                         electrode_measurements.append(np.array(X[idx[0]-50:idx[0]+250, electrode]))
#                 else:
#                     for electrode in range(8):
#                         if not (idx[0]-50 < 0 or idx[0]+250 > xlen):
#                             electrode_measurements[electrode] = np.vstack((electrode_measurements[electrode], np.array(X[idx[0]-50:idx[0]+250, electrode])))
    
#     means_arr = [np.mean(i, axis = 0) for i in electrode_measurements]
#     print(means_arr[0].shape)
    
# #     means_arr = np.array(means_arr)
# #     n = means_arr.T
#     graph_axis = [i/250 for i in range(-50, 250) ]
    
# #     figure, axis = plt.subplots(2, 4)
# #     for i in range(4):
# #         axis[0,i].plot(graph_axis,n[i])
# #         y_lab = f'Channel: {i}'
# #         axis[0,i].ylabel(y_lab)
# #         axis[0,i].xlabel(time)
# #         axis[0,i].set_title(f"Channel: {i}")
    
# #     for i in range(4, n.shape[0]):
# #         axis[0,i].plot(graph_axis,n[i])
# #         y_lab = f'Channel: {i}'
# #         axis[0,i].ylabel(y_lab)
# #         axis[0,i].xlabel(time)
# #         axis[0,i].set_title(f"Channel: {i}")
        
    
#     for i in range(len(means_arr)):
#         plt.figure(facecolor=(1, 1, 1))
#         plt.plot(graph_axis,means_arr[i])
#         y_lab = f'Channel: {i}'
#         plt.title(f'Channel: {i}')
#         plt.ylabel(y_lab)
#         plt.xlabel(time)
#         plt.savefig(f'Channel-{i}.png')
#         plt.show()
    
#     print("haha")
#     return X_filtered, Y, T, F